In [1]:
import pickle

rec_contex = {
    "가성비를 중시하는 고객" : "빅세일 기간에 최대 60% 할인이 이루어진다는 사실을 강조할 수 있음",
    "그린티씨드크림을 추천하는 경우" : "대용량 구매 시 1+1 혜택이 있음",
    "여행이나 야외 활동을 많이 하는 고객" : "작은 용량의 제품을 추천하는 것이 권장됨",
    "그린티씨드세럼을 추천하는 경우" : "리필을 할 수 있음을 강조할 수 있음",
    "트렌드에 민감한 고객" : "구매 시 '쿠키 키링 파우치' 증정을 강조할 수 있음",
    "카페를 좋아하는 고객" : "연말에 화장품을 구매하는 것을 강조할 수 있음"
}

with open(file='recommended_contents.pickle', mode='rb') as f:
    rc = pickle.load(f)

cp = {
    "그린티씨드세럼" : "대한민국 판매 1위 수분에센스. 단숨에 속건조 해결",
    "그린티씨드크림" : "외부자극으로 성난 피부 수분 진정. 재구매율 1위. 써 본 사람은 다 아는 확실한 수분 진정 효과. 수분을 빈틈 없이 꽉 채워 화장이 잘 먹는 수분크림",
    "레티놀시카앰플" : "국민 트러블 앰플. 갓 생긴 트러블부터 모공, 잡티, 흔적까지 트러블 끝장 케어. 반복되는 트러블 고리의 악순환을 끊는 #레티놀시카. 트러블, 모공, 잡티, 흔적 고민 지우고 깨끗하고 매끈매끈한 깐달걀 피부결",
    "레티놀PDRN앰플" : "모공결부터 속광까지 #바르는 스킨 부스터. 매끈한 모공결 & 촘촘한 피부광의 스킨부스터 앰플. 주름, 모공, 탄력 복잡한 피부 고민 이거 하나로 끝! 당신의 피부를 전방위적으로 개선합니다."
}

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from collections import Counter
import ast
from random import shuffle

warnings.filterwarnings('ignore')
tqdm.pandas()

import requests
import base64
import json

# Azure GPT API 설정
GPT4V_ENDPOINT = "https://apeus-cst-openai.openai.azure.com/openai/deployments/gpt4o/chat/completions?api-version=2024-02-15-preview"
GPT4V_KEY = "fb73c5cdb51d4167b9eb41345d059bfe"

# 헤더 설정
headers = {
    "Content-Type": "application/json;",
    "api-key": GPT4V_KEY,
}

def respond(payload):
    response = requests.post(GPT4V_ENDPOINT, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
        
    else:
        return f"Error {response.status_code}: {response.text}"

In [3]:
internal_df = pd.read_csv("internal_characteristics.csv", encoding = "utf-8-sig", index_col = 0)
internal_df["focus_item"] = internal_df["focus_item"].apply(lambda x : ast.literal_eval(x))

external_df = pd.read_csv("external_characteristics.csv", encoding = "utf-8-sig", index_col = 0)
external_df["external_item"] = external_df["external_item"].apply(lambda x : ast.literal_eval(x))

In [4]:
find = re.compile('그린티씨드세럼|그린티씨드크림|레티놀시카앰플|레티놀PDRN앰플|비타C엔자임세럼')
internal_df["recommended_item"] = internal_df["recommended_item"].apply(lambda x : find.search(x.replace(" ", ""))[0])
external_df["recommended_item"] = external_df["recommended_item"].apply(lambda x : find.search(x.replace(" ", ""))[0])

In [5]:
already_suggested = []

system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
user1 = """
나는 너에게 (1) 고객의 특징, (2) 추천하는 제품, (3) 추천하는 제품의 특징, (4) 추천의 이유 (5) 추천 맥락, (6) 피해야 할 문구 스타일들을 제공해 줄거야.
아래 포맷과 조건, 예시를 따라서, 고객에게 이 제품을 추천하는 문구를 2-3 문장으로 디자인해줘.
피해야할 문구 스타일들과 가급적이면 다른 스타일의 문구를 만들어줘. 없으면 그냥 만들어도 돼.
포맷: (1) 고객의 특징에 따른 맥락 언급하기. (2) 고객의 맥락에 의거해서 제품 추천하기
조건: 고객의 특징은 너무 세세하게 말해선 안 돼. 기본적으로 2문장을 생성하되, 사용해야 할 정보가 많아지면 3문장을 생성할 수 있어.
예시: [가성비 있는 피부결 관리템을 찾으시나요? 순한 성분으로 속건조를 해결하는 그린티씨드크림을 만나보세요!;
      바쁜 일상, 간단하지만 확실한 안티에이징 케어가 필요하다면? 다양한 피부 고민을 동시에 해결하는 레티놀PDRN앰플을 만나보세요!]
"""
assistant1 = "알겠습니다. 정보를 제공해 주세요."

user2 = f"""
(1) 고객의 특징: 바쁜 회사 생활 속에서도 건강한 식습관을 유지하는 고객.
(2) 추천하는 제품: 그린티씨드세럼
(3) 추천하는 제품의 특징: {cp["그린티씨드세럼"]}
(4) 추천의 이유: 이 고객군은 자연을 사랑하고 건강한 생활 습관을 유지하려고 노력하는 사람들이기 때문에, 천연 성분이 포함된 순하고 촉촉한 제품이 어울릴 것으로 예상됩니다. 그린티씨드세럼은 수분 공급과 순한 성분 덕분에 민감한 피부에도 적합하며, 바쁜 일상 속에서도 빠르게 흡수되어 사용하기 편리합니다.
(5) 추천 맥락: {rec_contex}
(6) 피해야 할 문구 스타일들: {already_suggested}
"""

assistant2 = "칼퇴근 후 배달 음식 대신 그린티 어때요? 자극받은 피부 진정시켜주는 그린티 씨드크림을 만나보세요"

user3 = f"""
(1) 고객의 특징: 외국과 국내 여행에 관심이 많고, 피부와 뷰티 제품에도 신경을 쓰며, 다양한 혜택과 프로모션에 민감하게 반응하는 고객.
(2) 추천하는 제품: 그린티씨드크림
(3) 추천하는 제품의 특징: {cp["그린티씨드크림"]}
(4) 추천의 이유:  이 고객군은 여행을 좋아하고 피부와 뷰티 제품에 관심이 많아 위생적이고 휴대가 간편한 제품이 필요합니다. 그린티씨드크림은 튜브형 포장으로 여행 시 휴대하기 편리하며 피부에 강력한 수분을 공급해줍니다.
(5) 추천 맥락: {rec_contex}
(6) 피해야 할 문구 스타일들: {already_suggested}
"""

assistant3 = "연말 나들이에는 꼭 데려가세요. 휴대하기 편한 강력한 수분 공급템, 그린티씨드크림을 만나보세요"

In [6]:
def generate_message_internal(internal_df):
    internal_result = []
    for i in tqdm(internal_df.index):
        try:
            already_suggested = list(internal_df[internal_df.columns[7:]].loc[i])
        except:
            already_suggested = []

        temp = internal_df["concern"].loc[0].strip().split(".")
        shuffle(temp)
        
        user4 = f"""
        (1) 고객의 특징: {". ".join(temp).strip()}
        (2) 추천하는 제품: {internal_df["recommended_item"].loc[i]}
        (3) 추천하는 제품의 특징: {rc[internal_df["recommended_item"].loc[i]]}
        (4) 추천의 이유: {internal_df["reason_of_recommendation"].loc[i]}
        (5) 추천 맥락: {rec_contex}
        (6) 피해야 할 문구 스타일들: {already_suggested}
        """
    
        payload = {
            "messages": [
                {"role": "system", "content": system1},
                {"role": "user", "content": user1},
                {"role": "assistant", "content": assistant1},
                {"role": "user", "content": user2},
                {"role": "assistant", "content": assistant2},
                {"role": "user", "content": user3},
                {"role": "assistant", "content": assistant3},
                {"role": "user", "content": user4}]}
    
        decision = True
        while decision:
            try:
                internal_result.append(respond(payload))
                decision = False
            except:
                pass
    return internal_result

In [7]:
def generate_message_external(external_df):
    external_result = []
    for i in tqdm(external_df.index):
        try:
            already_suggested = list(external_df[external_df.columns[9:]].loc[i])
        except:
            already_suggested = []

        temp = external_df["concern"].loc[0].strip().split(".")
        shuffle(temp)
            
        user4 = f"""
        (1) 고객의 특징: {". ".join(temp).strip()}
        (2) 추천하는 제품: {external_df["recommended_item"].loc[i]}
        (3) 추천하는 제품의 특징: {rc[external_df["recommended_item"].loc[i]]}
        (4) 추천의 이유: {external_df["reason_of_recommendation"].loc[i]}
        (5) 추천 맥락: {rec_contex}
        (6) 피해야 할 문구 스타일들: {already_suggested}
        """
    
        payload = {
            "messages": [
                {"role": "system", "content": system1},
                {"role": "user", "content": user1},
                {"role": "assistant", "content": assistant1},
                {"role": "user", "content": user2},
                {"role": "assistant", "content": assistant2},
                {"role": "user", "content": user3},
                {"role": "assistant", "content": assistant3},
                {"role": "user", "content": user4}]}
    
        decision = True
        while decision:
            try:
                external_result.append(respond(payload))
                decision = False
            except:
                pass
                
    return external_result

In [8]:
for i in range(2):
    internal_df[f"message{i}"] = generate_message_internal(internal_df)
    external_df[f"message{i}"] = generate_message_internal(external_df)

# internal_df.to_csv("internal_characteristics.csv", encoding = "utf-8-sig")
# external_df.to_csv("external_characteristics.csv", encoding = "utf-8-sig")

100%|██████████| 97/97 [04:16<00:00,  2.64s/it]


In [18]:
internal_df

,internal_cluster,focus_item,characteristics,concern,recommended_item,confidence,reason_of_recommendation,message0,message1,message2,message3,message4
0,0-0,"[131170000, 131171318, 131173347]","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...","자연스럽고 매혹적인 눈매와, 매끈하고 깨끗한 피부를 원함. 여름철 습하고 더운 날씨...",레티놀PDRN앰플,80%,"이 고객군은 매끈하고 깨끗한 피부를 원하며, 여름철에도 지속력 있는 메이크업을 추...",여름철에도 무너지지 않는 메이크업을 원하시나요? 피부를 탄력 있고 매끈하게 만들어주...,"더운 여름, 무너지지 않는 피부를 완성하고 싶다면 레티놀PDRN앰플을 사용해 보세요...",활동적인 라이프스타일 속에서도 밝고 생기 있는 피부를 원한다면? 피부 톤 개선과 촉...,활동적인 라이프스타일 속에서도 맑고 생기있는 피부를 원하시나요? 비타C엔자임세럼은 ...,활동적인 라이프스타일에도 건강한 피부를 원하시나요? 피부 톤 개선과 수분 공급을 한...
1,1-0,"[131172564, 131173041]","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개...",활기찬 하루를 보내면서도 피부에 자연스러운 광채와 탄력을 유지하고자 하는 열정적이고...,레티놀PDRN앰플,90%,"이 고객군은 피부의 광채와 탄력 유지에 대한 목표가 확고하고, 종합적인 안티에이징...",여름철에도 무너지지 않는 완벽한 메이크업을 원하시나요? 피부 탄력 향상과 주름 개선...,"여름철, 무너지지 않는 매끈한 피부를 원하시나요? 피부 탄력과 결 개선을 돕는 레티...","온종일 매끄럽고 탄력 있는 피부를 원하시나요? 피부 톤 개선과 보습, 탄력 향상을 ...",온종일 매끄럽고 탄력 있는 피부를 원하시나요? 톤 개선과 촉촉한 보습을 제공하는 레...,하루 종일 매끈하고 탄력 있는 피부를 원하시나요? 피부 톤 개선과 탄력 향상에 탁월...
2,10-0,"[131174965, 131172564]","지구 환경을 생각하면서도 자신의 피부 건강과 외모를 꼼꼼히 관리하는 고객. 0번, ...",피부의 탄력을 높이고 젊고 생기 넘치는 모습을 유지하고 싶어하는 고객. 다양한 활동...,레티놀PDRN앰플,90%,이 고객군은 피부의 탄력과 생기를 높이고 자극을 최소화하면서도 전반적인 피부 개선...,여름철에도 깨끗하고 매혹적인 눈매와 완벽한 피부를 원하시나요? 레티놀PDRN앰플로 ...,여름철 습기와 더위에도 흔들림 없는 메이크업을 원하시나요? 레티놀PDRN앰플로 강력...,스트레스로 인한 피부 순환 문제 걱정되시나요? 레티놀PDRN앰플이 피부 톤을 개선하...,피부 탄력과 영양이 걱정이라면? 레티놀PDRN앰플로 건강하고 생기있는 피부 케어를 ...,피부 순환과 탄력 걱정되시나요? 단단한 피부 탄력과 복합적인 문제를 해결해주는 레티...
3,10-2,"[111770997, 111506404]","자연 원료와 순면 제품을 선호하며, 피부와 자신을 소중하게 관리하는 고객. 0번, ...","자신의 피부를 부드럽게 감싸고, 자연의 순한 손길로 다독이는 것을 좋아하는 고객. ...",그린티씨드크림,90%,이 고객군은 피부의 민감성을 줄이고 자연의 순한 손길로 피부를 다독이는 것을 좋아...,여름철 더위와 마스크 속에서도 무너지지 않는 메이크업을 원하시나요? 피부에 순하게 ...,"습하고 더운 날씨에도 매끈한 메이크업을 유지하고 싶다면, 그린티씨드크림을 추천합니다...",새로운 모험을 즐기며 내 피부도 소중하게 챙기고 싶다면? 민감한 피부에도 자극 없이...,새로운 경험을 찾으세요? 여행 중에도 자극 없이 피부를 촉촉하게 유지하는 그린티씨드...,새로운 모험 속에서도 민감한 피부를 자극 없이 관리하고 싶으신가요? 피부를 진정시키...
4,11-0,"[131172879, 131174282]",자연 친화적인 성분과 제품을 선호하며 자신만의 고유한 향기와 스타일을 중요시하는 고...,섬세하고 감각적인 아름다움을 강조하는 고객. 주기적인 스트레스와 외부 환경으로 인한...,그린티씨드크림,85%,이 고객군은 주기적인 스트레스와 외부 환경으로 인해 건조함과 민감성을 겪고 있습니...,여름철 뜨거운 날씨와 마스크에도 무너지지 않는 매끈한 피부를 원하시나요? 그린티씨드...,여름철 습한 날씨와 마스크에도 무너지지 않는 피부를 원하시나요? 그린티씨드크림으로 ...,화이트닝 관리가 필요하신가요? 부드러운 발림성과 민감한 피부에도 사용 가능한 레티놀...,피부가 민감하지만 세련된 매력을 한층 더하고 싶으신가요? 순하고 효과적인 레티놀시카...,"세련된 취향과 고유의 매력을 표현하고 싶다면, 순하면서도 강력한 레티놀 시카 앰플을..."
5,11-1,"[111770997, 111506404, 111506405]","이 고객은 다양한 피부 고민을 해결하고 자주 화장솜 등으로 피부를 관리하며, 철저한...","자주 돌아다니면서 새로운 환경에 노출되며, 다양한 장소에서 활동하는 스타일로, 지속...",레티놀PDRN앰플,85%,"이 고객군은 여러 가지 피부 고민을 효과적으로 해결하고자 하며, 다양한 상황에서 ...",여름철 무더위에도 깨끗한 피부를 원하시나요? 다양한 피부 고민을 해결하는 레티놀PD...,여름철에도 무너지지 않는 깨끗한 피부를 원하시나요? 다양한 피부 고민을 전방위적으로...,"끊임없이 새로운 곳을 탐험하면서도 꼼꼼한 자기관리를 실천하는 당신께, 그린티씨드세럼...","끊임없이 새로운 곳을 탐험하면서도 자기관리에 철저한 당신에게, 그린티씨드세럼을 추천...",새로운 곳을 탐험하면서도 꼼꼼한 자기관리를 실천하는 당신에게 그린티 씨드 세럼을 추...
6,12-0,"[131174283, 131171020, 131173425]","피부를 건강하고 자연스럽게 유지하려는 신념을 가진 고객으로, 피부의 생기와 촉촉함을...","오랜 시간에도 생기를 잃지 않는 피부를 원하고, 항상 촉촉하게 유지되는 피부를 추구...",그린티씨드세럼,80%,이 고객군은 피부의 촉촉함을 항상 유지하고 싶어하며 자연스러운 상태를 중시합니다....,"여름에도 산뜻한 피부를 유지하고 싶으신가요? 자연스러운 매력을 더해주는 보습 세럼,...",여름철 무더위에도 무너지지 않는 메이크업을 원하시나요? 속건조를 단숨에 해결해주는 ...,당신의 피부에 건강한 생기를 더하고 싶으신가요? 자극 없이 촉촉함을 유지시켜주는 그...,피부에 생기를 더하고 촉촉함을 유지하고 싶으신가요? 천연 성분의 그린티씨드세럼이 일...,"일상에서 자외선도, 피부 자극도 잊고 싶으신가요? 끈적임 없이 촉촉하게 흡수되어 피..."
7,14-0,"[131172563, 131170641, 131175032]",자신만의 피부 관리와 세밀한 메이크업 루틴을 통해 완벽한 외모를 유지하려는 디테일한...,끊임없이 변화하는 환경 속에서도 완벽한 외모를 유지하고 싶은 열정을 지닌 고객. 피...,레티놀PDRN앰플,90%,"이 고객군은 끊임없이 변화하는 환경 속에서도 완벽한 외모를 유지하고자 하며, 다재...","습하고 더운 날씨에도 무너지지 않는 메이크업을 원하시나요? 피부 톤 개선, 주름 감...",여름철 습한 날씨와 마스크에도 완벽한 피부를 원하시나요? 다양한 효과를 자랑하는 레...,"활동적인 라이프스타일로 언제나 완벽한 피부를 원하신다면, 레티놀PDRN앰플을 추천합...","활동적인 라이프스타일 속에서도 완벽한 피부 관리, 레티놀PDRN앰플과 함께하세요. ...","외부 활동에도 언제나 완벽한 피부를 유지하고 싶으신가요? 탄력 향상, 피부 톤 개선..."
8,15-2,"[131171021, 131173502, 131172564]","자연스럽고 건강한 피부 관리를 중요하게 생각하며, 자외선 차단과 노화 방지를 위해 ...",끊임없이 활동하며 매일의 다양하고 바쁜 일상 속에서도 피곤한 티 없이 건강하고 자연...,레티놀PDRN앰플,90%,이 고객군은 바쁜 일상 속에서도 피부의 전반적인 건강과 노화를 방지하려는

In [19]:
external_df

,Unnamed: 0,internal_cluster_num,external_cluster_num,external_item,characteristics,internal_characteristics,concern,recommended_item,confidence,reason_of_recommendation,message0,message1,message2,message3,message4
0,0,0-0,0.0,"[PER_OTT 상, PER_MDLV 상]","시간과 에너지가 부족해 편리한 배달 음식을 주로 소비하지만, OTT 서비스를 통해 ...","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...",바쁜 일상 속에서도 자신의 아름다움을 놓치지 않으려는 고객이에요. 지속되는 높은 생...,레티놀시카앰플,85%,이 고객군은 활발한 생활 리듬과 다양한 역할을 소화하면서 트러블과 모공 관리에 특...,"피부 트러블과 모공 관리에 민감하시다면, 강력한 보습과 진정 효과를 선사할 레티놀시...",활발한 일상 속에서도 피부 트러블을 관리하고 싶으신가요? 순한 성분으로 피부 진정을...,빈틈없는 피부 관리가 필요하다면? 다양한 환경에서도 빠르고 편리하게 사용할 수 있는...,"여행 다니느라 바쁜 당신, 피부 관리 소홀히 하지 마세요. 촉촉하게 속건조를 잡아주...","여행 중에도 빼놓을 수 없는 피부 관리, 그린티씨드세럼으로 간편하게 해결해 보세요...."
1,1,0-0,1.0,"[PER_FRN_TRV 상, PER_DOM_TRV 상, PER_PRC_SNSV 상]","해외 여행에도 많은 관심을 가지고 있지만, 가격에 민감하여 합리적인 가격의 국내 여...","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...","여러 새로운 경험을 추구하고, 활동적인 라이프스타일을 즐기는 이 고객은 피부에 다양...",레티놀PDRN앰플,80%,"이 고객군은 활동적인 라이프스타일로 인해 모공과 피부결 관리가 중요하며, 자주 건...","활동적인 라이프스타일 속에서도 피부 고민, 놓칠 수 없죠? 트러블을 빠르게 진정시키...","활동적인 라이프스타일에도 지친 피부와 모공 관리를 놓칠 수 없다면, 레티놀PDRN앰...",다양한 환경에서도 피부 관리가 필요하시죠? 자극 없이 피부 톤을 환하게 잡아주는 레...,활동적인 라이프를 즐기면서도 비용도 중요하신가요? 피부 톤 개선과 트러블 완화에 효...,여행 속에서도 피부 트러블과 자극이 걱정되시나요? 보습과 트러블 완화는 물론 피부 ...
2,2,0-0,2.0,"[JOB 02_회사원, INCOME 01_중하계층, LFSTG 01.미혼추정]","금전적 여유는 많지 않지만, 바쁜 회사 생활과 혼자만의 시간을 보내는 데 집중하는 ...","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고...",이 고객은 바쁜 일상 속에서도 자기만의 시간을 소중히 여기는 사람입니다. 금전적 여...,레티놀시카앰플,80%,이 고객군은 바쁜 일상 속에서도 자기관리를 중요시하며 특히 스트레스로 인한 피부 ...,"여름철 활동적인 라이프스타일을 위해, 강력한 트러블 완화 효과를 제공하는 레티놀 시...",지친 일상 속에서도 맑고 깨끗한 피부를 원하시나요? 트러블 완화와 피부 톤 개선에 ...,"바쁜 도심 속에서도 건강한 피부를 유지하고 싶다면, 레티놀시카앰플을 만나보세요. 이...",바쁜 일상 속에서도 트러블 관리가 필요하신가요? 짧은 시간에 피부 톤 개선과 보습을...,바쁜 도심 속에서도 피부 관리 포기하지 마세요. 트러블 완화와 피부 톤 개선에 효과...
3,3,1-0,0.0,"[PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상]","가격에 민감하지만, 편리함을 추구하며 배달 식품을 자주 먹으며 OTT 서비스를 즐기...","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개...",활동성이 높으면서도 세밀한 자기 관리를 중요하게 여기는 이 고객은 라이프스타일 속 ...,레티놀PDRN앰플,90%,"이 고객군은 활동성이 높고 다양한 외부 요인에 피부가 쉽게 영향을 받기 때문에, ...","여름철 방수 메이크업을 선호하신다면, 신속하게 피지와 트러블을 케어할 수 있는 레티...","피부 트러블과 피지 관리에 신경 쓰인다면, 레티놀 PDRN 앰플을 사용해보세요. 올...","변화 많은 하루 중에도 피부의 건강과 젊음을 유지하고 싶다면, 레티놀PDRN앰플을 ...",매일 반복되는 스트레스 속에서도 피부의 젊음을 유지하고 싶으신가요? 주름 개선과 피...,끊임없이 변화하는 라이프스타일 속에서도 피부의 건강과 젊음을 유지하고 싶으신가요? ...
4,4,1-0,1.0,"[PER_DOM_TRV 상, PER_PRC_SNSV 상]","가성비를 중요하게 생각하며 주로 국내 여행에 관심을 두고 있는 고객. 1번, 2번 정보.","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개...","활동적이고 잦은 이동을 하는 라이프스타일을 가진 당신, 여행 중에도 간편하게 들고 ...",레티놀PDRN앰플,90%,이 고객군은 활동적이고 이동이 잦아 간편하면서 고효율적인 스킨케어 제품을 찾고 있...,"여행이 잦고 다양한 피부 케어를 원하시나요? 주름, 모공, 탄력 등 복합적인 피부 ...","여행과 바쁜 일상 속에서도 세련된 피부를 유지하고 싶으신가요? 주름, 모공, 탄력까...",활발한 일상 속에서도 피부 관리를 놓치고 싶지 않으신가요? 피부 탄력과 주름 개선을...,"어디서든 피부를 신경 쓰시는 당신께, 레티놀 PDRN 앰플을 추천합니다. 주름 개선...","언제 어디서든 피부 건강을 빠르게 챙기고 싶다면, 레티놀 PDRN 앰플을 추천드려요..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,7-1,4.0,"[PER_FRN_TRV 상, PER_DOM_TRV 상]",다양한 문화 체험과 여가 활동을 즐기며 국내외 여행에 많은 관심을 갖고 있는 고객....,자주 트러블이 발생하는 피부 고민을 다양한 제품을 통해 체계적으로 관리하고 있는 고...,"다채로운 생활 속에서 다양한 경험을 중시하고 있으며, 번잡한 일상에서 벗어나 자연스...",레티놀시카앰플,80%,이 고객군은 번잡한 일상 속에서 피부 스트레스를 줄이고 트러블이 발생할 경우 즉각...,바쁜 일상 속 지친 피부 관리를 고민 중이신가요? 즉각적인 트러블 대응과 수분 공급...,피부 트러블이 걱정되시나요? 높은 생활 리듬 속에서도 즉각적인 케어와 보습을 제공하...,"늘 새로운 도전을 즐기는 당신, 레티놀시카앰플로 피부 관리도 업그레이드하세요. 피부...","끊임없이 도전하며 세상을 탐험하는 당신, 민감해진 피부도 놓칠 수 없죠. 트러블 완...",늘 도전과 열정으로 가득 찬 당신에게 필요한 레티놀시카앰플을 추천드립니다. 다양한 ...
93,93,7-1,5.0,"[LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상]",바쁜 직장 생활과 자녀 돌봄의 이중 부담 속에서 편리함을 중시하여 배달 식품을 자주...,자주 트러블이 발생하는 피부 고민을 다양한 제품을 통해 체계적으로 관리하고 있는 고...,강한 책임감과 열정으로 매일을 살아가는 고객. 하지만 바쁜 일상 속에서 피부 관리에...,레티놀시카앰플,85%,"이 고객군은 바쁜 일상 속에서도 실용적이고 효과적인 스킨케어 솔루션을 원하며, 피...",바쁜 일상에서도 피부 트러블 해결이 필요하신가요? 즉각적인 피부 개선과 보습 효과를...,늘 바쁜 일정을 소화하면서도 세련된 피부를 유지하고 싶으신가요? 레티놀 시카 앰플로...,피부 트러블로 고민 중이신가요? 레티놀 시카 앰플로 트러블을 완화하고 매끈한 피부를...,"자주 발생하는 피부 트러블로 고민이시라면, 레티놀시카앰플을 추천해드립니다. 트러블 ...",피부 트러블 관리 고민 중이신가요? 순한 성분으로 트러블을 완화시키며 보습과 피부 ...
94,94,8-0,2.0,"[JOB 02_회사원, INCOME 01_중하계층, LFSTG 01.미혼추정]",